In [ ]:
import torch
from mergoo.compose_experts import ComposeExperts

In [ ]:
config = \
{
    "model_type": "llama",
    "num_experts_per_tok": 2,
    "experts":[
        {
            "expert_name" : "base_expert",
            "model_id" : "../../llm/models/hf/old/2jaA"
        },
        {
            "expert_name" : "expert_1",
            "model_id" : "../../llm/models/hf/old/2jaA"
        },
        {
            "expert_name" : "expert_2",
            "model_id" : "../../llm/models/hf/old/1code"
        }
    ],
    "router_layers":[
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
}

In [ ]:

model_id = "merged_models/mergoo_llama_test"

In [ ]:
#expertmerger = ComposeExperts(config, torch_dtype=torch.float16)
#expertmerger.compose()
#expertmerger.save_checkpoint(model_id)

In [ ]:
# load the composed checkkpoint
import torch
from mergoo.models.modeling_llama import LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained(
    model_id, 
    device_map="auto", 
    torch_dtype=torch.bfloat16,
)# 'gate' / router layers are untrained hence loaded warning would appeare for them



# train only router (gating) layers
n_weights, n_router_weights  = 0,0
for name, weight in model.named_parameters():
    if "gate" not in name:
        weight.requires_grad_(False)
        n_router_weights += 1
    n_weights += 1
n_weights, n_router_weights